In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

# Mars Satellite Flight:

### What Is the minimum energy needed to launch a rocket for a Satellite to reach orbit around Mars

By Robin Graham-Hayes and Theo Johnson

links: 
https://en.wikipedia.org/wiki/McDonnell_Douglas_DC-X
http://www.astronautix.com/d/dc-x.html
https://en.wikipedia.org/wiki/Thrust

Situation:

In the future, it is possible that space missions will be launched from Mars. Not only is this more convenient because it is further into the solar system than Earth, it also has a smaller gravity field. This could lead to the emergence of one-stage rockets, namely ion rockets, as they require less energy to launch.

Question:

This presented us with the question: What launch conditions for an ion rocket on Mars requires the minimum amount of energy to result in an orbit around Mars.

Method:

A rocket would require the least energy when it uses the least amount of thrust. To find where the rocket needs the lowest thrust, we decided to sweep the launch angle and thrust in order to find what scenario produces an orbiting rocket with the lowest thrust.


Creating our Model:

First, we started my making our system and state. For our state, we started with the rocket stationary on the surface of Mars meaning our velocities and x were 0 and our y was 3389000 meters, the radius of Mars. This also made our gravitational force equivalent to the gravitational constant of Mars. Our system included the mass of the rocket, obtained from the Wikipedia page on the McDonnell Douglas DC-X, an one-stage rocket. It also includes the mass of Mars, the gravitational constant on Mars, and the C_d value of the rocket.

In [3]:
# ABD: thrust and thrust_hat should not be part of your state.  
# they are inputs to the model, not state variables


init = State(
    x=0,
    y=3389000, #m,
    vx=0,
    vy=0,
    thrust_hat=Vector(8,100).hat(),
    thrust=Vector(8,100).hat()*1
)   



init.thrust_hat.angle

<Quantity(1.4909663410826592, 'radian')>

In [4]:
system = System(
    init=init,   
    mars_mass= 6.39E23, #kg
    mass_rocket=18900, #kilograms
    g_0_mars= 3.711, #m/s^2
    area=np.pi * (4.1/2)**2,
    C_d= .5,
    G=6.674E-11,
    t_0 = 0,
    t_end=24000
    )

,values
init,x ...
mars_mass,6.39e+23
mass_rocket,18900
g_0_mars,3.711
area,13.2025
C_d,0.5
G,6.674e-11
t_0,0
t_end,24000


For our drag force function, we used the same equation as we did in our notebook. It takes in the atmospheric pressure, the velocity, the C_d, and the area and returns the force from drag which is used later to calculate the total external force on the rocket.

In [5]:
def drag_force(state, V, system):
  
    unpack(system)
    mag = -pressure(state,system) * V.mag**2 * C_d * area / 2
    direction = V.hat()
    f_drag = mag * direction
    return f_drag

To find our gravitational force, we used the universal_gravitation function from our orbital mechanics notebook, replacing the sun with Mars and Earth with the rocket. This returns a force which is used to find the total external force on the rocket.

In [6]:
def universal_gravitation(state, system):
    """Computes gravitational force.
    
    state: State object with distance r
    system: System object with m1, m2, and G
    """
    x, y, vx, vy, thrust_hat,thrust= state
    unpack(system)
    
    v = Vector(vx,vy)
    d = Vector(x,y)
   
    force_mag = G * mass_rocket * mars_mass / d.mag**2
    
    force = d.hat() * force_mag
    
    return force

Our pressure function updates pressure according to the Mars atmospheric pressure equation found on NASA's website. It takes in the rocket's distance from the center of Mars and returns the pressure at that height.

In [7]:
def pressure(state,system):
    x, y, vx, vy, thrust_hat,thrust = state
    unpack(system)
    d = Vector(x,y)
    p=.699*exp(-.00009*d.mag)
    return p

In [8]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy, thrust_hat,thrust = state
    unpack(system)

    V = Vector(vx, vy)    
    a_drag = drag_force(state, V, system) / mass_rocket
    a_grav = -universal_gravitation(state, system)
    a_rocket = Vector(thrust.x, thrust.y).mag / mass_rocket
    
    a = a_grav + a_drag + a_rocket
    
    
    # ABD: whatever state variables you have, you have to return the
    # corresponding derivatives.
    return vx, vy, a.x, a.y

Our event function causes a stop when the rocket is perpendicular to Mars, this is when the rocket would be in position to cut off thrust and be in orbit. 

In [9]:
def event_func(state, t, system):
    unpack(system)
    x, y, vx, vy, thrust_hat,thrust = state
    
    V=Vector(vx,vy)
    
    return V.angle - (((pi/2)-1.4909663410826592) + universal_gravitation(state, system).angle)

In [10]:
event_func(init, 0, system)
slope_func(init, 0, system)

(0,
 0,
 <Quantity(5.2910052910052905e-05, 'dimensionless')>,
 <Quantity(-70178.76586926138, 'dimensionless')>)

In [0]:
  results1, details1 = run_ode_solver(system, slope_func, events=event_func, method='RK45')
details1

ValueError: ignored

In [0]:
plot(results1.x,results1.y)
decorate(xlabel='X position (meter)',
         ylabel='Y position (m)',
         title='Position during launch',
         legend=False)

NameError: ignored

In [0]:
plot(time,results1.a.angle)
decorate(xlabel='Time (s)',
         ylabel='Acceleration angle (radians)',
         title='Angle of Acceleration Vs. Time',
         legend=False)

Abstract: 

In this project, we asked: what is the optimal launch angle and thrust are to minimize energy when launching a rocket into orbit on Mars?

Configured for ModSimPy. Restarting kernel.
